## 7.10 数组

An array is implemented simply by storing the elements consecutively in memory. No
information about the dimensions of the array is stored. This makes the use of arrays in C
and C++ faster than in other programming languages, but also less safe. This safety
problem can be overcome by defining a container class that behaves like an array with
bounds checking, as illustrated in this example:

```cpp
// Example 7.15a. Array with bounds checking
template <typename T, unsigned int N> class SafeArray {
protected:
    T a[N]; // Array with N elements of type T
public:
    SafeArray() { // Constructor
        memset(a, 0, sizeof(a)); // Initialize to zero
    }
    int Size() { // Return the size of the array
        return N;
    }
    T & operator[] (unsigned int i) { // Safe [] array index operator
    if (i >= N) {
        // Index out of range. The next line provokes an error.
        // You may insert any other error reporting here:
        return *(T*)0; // Return a null reference to provoke error
    }
    // No error
    return a[i]; // Return reference to a[i]
}
```

More examples of container classes are given in www.agner.org/optimize/cppexamples.zip.

An array using the above template class is declared by specifying the type and size as
template parameters, as example 7.15b below shows. It is accessed with a square brackets
index, just as a normal array. The constructor sets all elements to zero. You may remove
the memset line if you don't want this initialization, or if the type T is a class with a default
constructor that does the necessary initialization. The compiler may report that memset is
deprecated. This is because it can cause errors if the size parameter is wrong, but it is still
the fastest way to set an array to zero. The [] operator will detect an error if the index is out
of range (see page 138 on bounds checking). An error message is provoked here in a
rather unconventional manner by returning a null reference. This will provoke an error
message in a protected operating system if the array element is accessed, and this error is
easy to trace with a debugger. You may replace this line by any other form of error
reporting. For example, in Windows, you may write FatalAppExitA(0,"Array index
out of range"); or better, make your own error message function.

The following example illustrates how to use SafeArray:
```cpp
// Example 7.15b
SafeArray <float, 100> list; // Make array of 100 floats
for (int i = 0; i < list.Size(); i++) { // Loop through array
    cout << list[i] << endl; // Output array element
}
```

An array initialized by a list should preferably be static, as explained on page 27. An array
can be initialized to zero by using memset:
```cpp
// Example 7.16
float list[100];
memset(list, 0, sizeof(list));
```

A multidimensional array should be organized so that the last index changes fastest:
```cpp
// Example 7.17
const int rows = 20, columns = 50;
float matrix[rows][columns];
int i, j; float x;
    for (i = 0; i < rows; i++)
        for (j = 0; j < columns; j++)
            matrix[i][j] += x;
```


This makes sure that the elements are accessed sequentially. The opposite order of the two
loops would make the access non-sequential which makes the data caching less efficient.

The size of all but the first dimension may preferably be a power of 2 if the rows are indexed
in a non-sequential order in order to make the address calculation more efficient:

```cpp
// Example 7.18
int FuncRow(int); int FuncCol(int);
const int rows = 20, columns = 32;
float matrix[rows][columns];
int i; float x;
for (i = 0; i < 100; i++)
    matrix[FuncRow(i)][FuncCol(i)] += x;
```

Here, the code must compute (FuncRow(i)*columns + FuncCol(i)) *
sizeof(float) in order to find the address of the matrix element. The multiplication by
columns in this case is faster when columns is a power of two. In the preceding example,
this is not an issue because an optimizing compiler can see that the rows are accessed
consecutively and can calculate the address of each row by adding the length of a row to
the address of the preceding row. 

The same advice applies to arrays of structure or class objects. The size (in bytes) of the
objects should preferably be a power of 2 if the elements are accessed in a non-sequential
order.

The advice of making the number of columns a power of 2 does not always apply to arrays
that are bigger than the level-1 data cache and accessed non-sequentially because it may
cause cache contentions. See page 89 for a discussion of this problem.